In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# ScipyOptimizeDriver

ScipyOptimizeDriver wraps the optimizers in *scipy.optimize.minimize*. In this example, we use the SLSQP optimizer to find the minimum of the Paraboloid problem.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src019", get_code("openmdao.test_suite.components.paraboloid.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_src019`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-9
prob.driver.options['disp'] = True

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')

prob.setup()

prob.set_val('x', 50.0)
prob.set_val('y', 50.0)

prob.run_driver()

In [ ]:
print(prob.get_val('x'))

In [ ]:
print(prob.get_val('y'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob.get_val('x'), 6.66666667, 1e-6)
assert_near_equal(prob.get_val('y'), -7.3333333, 1e-6)

## ScipyOptimizeDriver Options

In [ ]:
om.show_options_table("openmdao.drivers.scipy_optimizer.ScipyOptimizeDriver")

## ScipyOptimizeDriver Constructor

The call signature for the *ScipyOptimizeDriver* constructor is:

```{eval-rst}
    .. automethod:: openmdao.drivers.scipy_optimizer.ScipyOptimizeDriver.__init__
        :noindex:
```

## ScipyOptimizeDriver Option Examples

**optimizer**

    The “optimizer” option lets you choose which optimizer to use. ScipyOptimizeDriver supports all of the optimizers in scipy.optimize except for ‘dogleg’ and ‘trust-ncg’. Generally, the optimizers that you are most likely to use are “COBYLA” and “SLSQP”, as these are the only ones that support constraints. Only SLSQP supports equality constraints. SLSQP also uses gradients provided by OpenMDAO, while COBYLA is gradient-free. Also, SLSQP supports both equality and inequality constraints, but COBYLA only supports inequality constraints.

    Here we pass the optimizer option as a keyword argument.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])

prob.driver = om.ScipyOptimizeDriver(optimizer='COBYLA')

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')

prob.setup()

prob.set_val('x', 50.0)
prob.set_val('y', 50.0)

prob.run_driver()

In [ ]:
print(prob.get_val('x'))

In [ ]:
print(prob.get_val('y'))

In [ ]:
assert_near_equal(prob.get_val('x'), 6.66666667, 1e-6)
assert_near_equal(prob.get_val('y'), -7.3333333, 1e-6)

**maxiter**

    The “maxiter” option is used to specify the maximum number of major iterations before termination. It is generally a valid option across all of the available options.

In [ ]:


import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['maxiter'] = 20

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')

prob.setup()

prob.set_val('x', 50.0)
prob.set_val('y', 50.0)

prob.run_driver()



In [ ]:
print(prob.get_val('x'))

In [ ]:
print(prob.get_val('y'))

In [ ]:
assert_near_equal(prob.get_val('x'), 6.66666667, 1e-6)
assert_near_equal(prob.get_val('y'), -7.3333333, 1e-6)

**tol**

    The “tol” option allows you to specify the tolerance for termination.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['tol'] = 1.0e-9

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')

prob.setup()

prob.set_val('x', 50.0)
prob.set_val('y', 50.0)

prob.run_driver()


In [ ]:
print(prob.get_val('x'))


In [ ]:
print(prob.get_val('y'))

In [ ]:
assert_near_equal(prob.get_val('x'), 6.66666667, 1e-6)
assert_near_equal(prob.get_val('y'), -7.3333333, 1e-6)

## ScipyOptimizeDriver Driver Specific Options

Optimizers in *scipy.optimize.minimize* have optimizer specific options. To let the user specify values for these options, OpenMDAO provides an option in the form of a dictionary named *opt_settings*. See the *scipy.optimize.minimize* documentation for more information about the driver specific options that are available.

As an example, here is code using some *opt_settings* for the shgo optimizer:

In [ ]:
# Source of example: https://stefan-endres.github.io/shgo/

import numpy as np
import openmdao.api as om

size = 3  # size of the design variable

def rastrigin(x):
    a = 10  # constant
    return np.sum(np.square(x) - a * np.cos(2 * np.pi * x)) + a * np.size(x)

class Rastrigin(om.ExplicitComponent):

    def setup(self):
        self.add_input('x', np.ones(size))
        self.add_output('f', 0.0)
        
        self.declare_partials(of='f', wrt='x', method='cs')

    def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):
        x = inputs['x']
        outputs['f'] = rastrigin(x)

prob = om.Problem()
model = prob.model

model.add_subsystem('rastrigin', Rastrigin(), promotes=['*'])

prob.driver = driver = om.ScipyOptimizeDriver()
driver.options['optimizer'] = 'shgo'
driver.options['disp'] = False
driver.opt_settings['maxtime'] = 10  # seconds
driver.opt_settings['iters'] = 3
driver.opt_settings['maxiter'] = None

model.add_design_var('x', lower=-5.12*np.ones(size), upper=5.12*np.ones(size))
model.add_objective('f')
prob.setup()

prob.set_val('x', 2*np.ones(size))
prob.run_driver()


In [ ]:
print(prob.get_val('x'))

In [ ]:
print(prob.get_val('f'))

In [ ]:
assert_near_equal(prob.get_val('x'), np.zeros(size), 1e-6)
assert_near_equal(prob.get_val('f'), 0.0, 1e-6)

Notice that when using the shgo optimizer, setting the *opt_settings[‘maxiter’]* to None overrides *ScipyOptimizeDriver*’s *options[‘maxiter’]* value. It is not possible to set *options[‘maxiter’]* to anything other than an integer so the *opt_settings[‘maxiter’]* option provides a way to set the maxiter value for the shgo optimizer to None.